In [1]:
# Openfermionpyscf : Hamiltonian construction
import openfermion as of
from openfermionpyscf import run_pyscf

# pennylane : quantum circuit & simulation
import pennylane as qml

# pennylane numpy :autograd
from pennylane import numpy as np

# default numpy
import numpy as npy
import itertools

# helper function
from helper import *

import pprint
import pickle

In [2]:
# molecule setting
basis = 'sto-3g'
multiplicity = 1

bond_length=0.7
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., round(bond_length,3))),('H', (0., 0., round(2*bond_length,3))),('H', (0., 0., round(3*bond_length,3)))]
molecule = of.MolecularData(geometry,    basis,    multiplicity)
molecule = run_pyscf(molecule, run_fci=True)

print(geometry)

# molecule properties
num_spatial_orbitals=molecule.n_orbitals
num_qubits=molecule.n_qubits
num_electrons=molecule.n_electrons

print(" #Spatial orbitals (Full Problem) :",num_spatial_orbitals)
print(" #Qubits (Full Problem) :",num_qubits)
print(" #Electrons (Full Problem) :",num_electrons)

[('H', (0.0, 0.0, 0.0)), ('H', (0.0, 0.0, 0.7)), ('H', (0.0, 0.0, 1.4)), ('H', (0.0, 0.0, 2.1))]
 #Spatial orbitals (Full Problem) : 4
 #Qubits (Full Problem) : 8
 #Electrons (Full Problem) : 4


In [3]:
hamiltonian = molecule.get_molecular_hamiltonian()

# Fermionic Hamiltonian <-> JW Qubit Hamiltonian
Ham = of.jordan_wigner(hamiltonian)

# Data type tranformation  (openfermion <-> pennylane)
Ham_in=qml.import_operator(Ham)

# eigenvalues /eigenvectors (mixed particles number)

eigenValues, eigenVectors = np.linalg.eig(npy.round(qml.matrix(Ham_in),9).real)
idx = eigenValues.argsort()[::1]   
eigenValues = eigenValues[idx].real
eigenVectors = eigenVectors[:,idx].real

In [4]:
n_orbitals=hamiltonian.n_qubits
n_electrons=num_electrons

N_Op=qml.qchem.particle_number(n_orbitals)
N_Op=qml.matrix(N_Op).real

S_z_Op=qml.qchem.spinz(n_orbitals)
S_z_Op=qml.matrix(S_z_Op).real

S2_Op=qml.qchem.spin2(n_electrons,n_orbitals)
S2_Op=qml.matrix(S2_Op).real

In [5]:
ans_index=[]
ans_S=[]
ans_Sz=[]
for i in range(20):
    n_particles=np.round( npy.transpose(eigenVectors[:,i]) @ N_Op @ eigenVectors[:,i] )
    if n_particles==n_electrons:
        S2= npy.transpose(eigenVectors[:,i]) @ S2_Op @ eigenVectors[:,i]
        print('index :',i)
        print("Energy : ",eigenValues[i])
        print('S^2: ', np.round(S2))
        print("S: ", np.round(-0.5+ npy.sqrt(1/4+S2)))
        ans_S.append(np.round(-0.5+ npy.sqrt(1/4+S2)))
        print("Sz : ", npy.round((npy.transpose(eigenVectors[:,i]) @ S_z_Op @ eigenVectors[:,i]),4),'\n')
        ans_Sz.append(npy.round((npy.transpose(eigenVectors[:,i]) @ S_z_Op @ eigenVectors[:,i]),4))
        ans_index.append(i)
E_ans=[np.round(eigenValues[i],8) for i in ans_index]

index : 0
Energy :  -2.1069969152460515
S^2:  0.0
S:  0.0
Sz :  -0.0 

index : 3
Energy :  -1.6544708921467672
S^2:  2.0
S:  1.0
Sz :  -0.0 

index : 4
Energy :  -1.6544708911776693
S^2:  2.0
S:  1.0
Sz :  -1.0 

index : 5
Energy :  -1.6544708911776553
S^2:  2.0
S:  1.0
Sz :  1.0 

index : 8
Energy :  -1.3656526668905762
S^2:  0.0
S:  0.0
Sz :  0.0 

index : 11
Energy :  -1.2806823544758597
S^2:  2.0
S:  1.0
Sz :  -0.0 

index : 12
Energy :  -1.2806823544367756
S^2:  2.0
S:  1.0
Sz :  1.0 

index : 13
Energy :  -1.2806823544367725
S^2:  2.0
S:  1.0
Sz :  -1.0 

index : 14
Energy :  -1.1777858690619294
S^2:  -0.0
S:  0.0
Sz :  -0.0 



In [6]:
E_ans_sz=[]
for i in range(len(ans_index)):
    if ans_Sz[i]==0:
        E_ans_sz.append(np.round(eigenValues[ans_index[i]],8))

In [7]:
E_ans_sz[0:4]

[-2.10699692, -1.65447089, -1.36565267, -1.28068235]

### PYSCF

In [8]:
bond_length=0.7

In [9]:
import pyscf
from pyscf import gto
from pyscf import scf, mcscf
r=bond_length
molh4=' H 0 0 0.0; H 0 0 {0};H 0 0 {1}; H 0 0 {2}'.format(r,2*r,np.round(3*r,6))
print(molh4)
mol = gto.M(atom=molh4, basis="sto-3g",spin=0)
myhf = mol.RHF()

 H 0 0 0.0; H 0 0 0.7;H 0 0 1.4; H 0 0 2.1


In [10]:
cisolver = pyscf.fci.FCI(myhf.run())
cisolver.nstates=4
pyscf_ans=cisolver.run().kernel()[0]
pyscf_ans

converged SCF energy = -2.06919742280387


array([-2.10699692, -1.65447089, -1.36565267, -1.28068235])

In [11]:
# All particle conseving SDs
all_SDs=generate_binary_strings_with_counts(n_orbitals,n_electrons)

# Calculate SDs energies (Also can get SDs energies from molecule-integrals)
N_qubits=n_orbitals
dev_n = qml.device("lightning.qubit", wires=N_qubits)
@qml.qnode(dev_n,diff_method=None)
def SDs_energy(vector):
    qml.BasisState(np.array(vector), wires=range(N_qubits))
    return qml.expval(Ham_in)

In [12]:
All_SD_sets=dict()
for i in all_SDs:
    print("M: ",i)
    temp=[]
    for j in all_SDs[i]:
        SD_in=binary_string_to_list(j)
        print(' SD: ', SD_in,'Index: ',binatodeci(SD_in),'Energy of SD: ',np.round(npy.array(SDs_energy(SD_in)),8))
        temp.append([SD_in,binatodeci(SD_in),np.round(npy.array(SDs_energy(SD_in)),8)])
    All_SD_sets[i]=temp
    print('\n')

M:  0
 SD:  [1, 1, 1, 1, 0, 0, 0, 0] Index:  240 Energy of SD:  -2.06919742
 SD:  [1, 1, 1, 0, 0, 1, 0, 0] Index:  228 Energy of SD:  -1.47924595
 SD:  [1, 1, 1, 0, 0, 0, 0, 1] Index:  225 Energy of SD:  -0.44288624
 SD:  [1, 1, 0, 1, 1, 0, 0, 0] Index:  216 Energy of SD:  -1.47924595
 SD:  [1, 1, 0, 1, 0, 0, 1, 0] Index:  210 Energy of SD:  -0.44288624
 SD:  [1, 1, 0, 0, 1, 1, 0, 0] Index:  204 Energy of SD:  -0.8538914
 SD:  [1, 1, 0, 0, 1, 0, 0, 1] Index:  201 Energy of SD:  0.18445573
 SD:  [1, 1, 0, 0, 0, 1, 1, 0] Index:  198 Energy of SD:  0.18445573
 SD:  [1, 1, 0, 0, 0, 0, 1, 1] Index:  195 Energy of SD:  1.32995005
 SD:  [1, 0, 1, 1, 0, 1, 0, 0] Index:  180 Energy of SD:  -1.13085784
 SD:  [1, 0, 1, 1, 0, 0, 0, 1] Index:  177 Energy of SD:  -0.13283469
 SD:  [1, 0, 1, 0, 0, 1, 0, 1] Index:  165 Energy of SD:  0.36486666
 SD:  [1, 0, 0, 1, 1, 1, 0, 0] Index:  156 Energy of SD:  -0.53231567
 SD:  [1, 0, 0, 1, 1, 0, 0, 1] Index:  153 Energy of SD:  0.46769489
 SD:  [1, 0, 0, 1, 0

In [13]:
# number of eigen-energies you are interested in
k_number=4

# sort
inputstates=[x[0] for x in sorted(All_SD_sets[0],reverse=False,key=lambda x:(x[2],-x[1]))][0:k_number]
pprint.pprint(sorted(All_SD_sets[0],reverse=False,key=lambda x:(x[2],-x[1])))

 
print('\n','Reference states : \n',inputstates)

[[[1, 1, 1, 1, 0, 0, 0, 0], 240, -2.06919742],
 [[1, 1, 1, 0, 0, 1, 0, 0], 228, -1.47924595],
 [[1, 1, 0, 1, 1, 0, 0, 0], 216, -1.47924595],
 [[1, 0, 1, 1, 0, 1, 0, 0], 180, -1.13085784],
 [[0, 1, 1, 1, 1, 0, 0, 0], 120, -1.13085784],
 [[1, 1, 0, 0, 1, 1, 0, 0], 204, -0.8538914],
 [[1, 0, 0, 1, 1, 1, 0, 0], 156, -0.53231567],
 [[0, 1, 1, 0, 1, 1, 0, 0], 108, -0.53231567],
 [[1, 1, 1, 0, 0, 0, 0, 1], 225, -0.44288624],
 [[1, 1, 0, 1, 0, 0, 1, 0], 210, -0.44288624],
 [[1, 0, 1, 1, 0, 0, 0, 1], 177, -0.13283469],
 [[0, 1, 1, 1, 0, 0, 1, 0], 114, -0.13283469],
 [[0, 0, 1, 1, 1, 1, 0, 0], 60, -0.11968395],
 [[1, 1, 0, 0, 1, 0, 0, 1], 201, 0.18445573],
 [[1, 1, 0, 0, 0, 1, 1, 0], 198, 0.18445573],
 [[1, 0, 1, 0, 0, 1, 0, 1], 165, 0.36486666],
 [[0, 1, 0, 1, 1, 0, 1, 0], 90, 0.36486666],
 [[1, 0, 0, 1, 0, 1, 1, 0], 150, 0.44277983],
 [[0, 1, 1, 0, 1, 0, 0, 1], 105, 0.44277983],
 [[1, 0, 0, 1, 1, 0, 0, 1], 153, 0.46769489],
 [[0, 1, 1, 0, 0, 1, 1, 0], 102, 0.46769489],
 [[0, 0, 1, 1, 1, 0, 0, 

### Excitation Pools

In [14]:
active_orbitals=[x for x in range(n_orbitals)]

sz = npy.array( [0.5 if (i % 2 == 0) else -0.5 for i in range(len(active_orbitals))])

Allsingles=list(itertools.permutations(active_orbitals,2))
Alldoubles=list(itertools.permutations(active_orbitals,4))

##### $\Delta$ Sz=0 excitations

In [15]:
singles=[]
for r,p in Allsingles:
    # delta sz=0 generalized excitations
    if sz[r]-sz[p]==0:
        singles.append([r,p])
        print((r,p))

(0, 2)
(0, 4)
(0, 6)
(1, 3)
(1, 5)
(1, 7)
(2, 0)
(2, 4)
(2, 6)
(3, 1)
(3, 5)
(3, 7)
(4, 0)
(4, 2)
(4, 6)
(5, 1)
(5, 3)
(5, 7)
(6, 0)
(6, 2)
(6, 4)
(7, 1)
(7, 3)
(7, 5)


In [16]:
doubles = []
for p, q, r, s in Alldoubles:
    if sz[p] + sz[q] - sz[r] - sz[s] == 0:
        doubles.append([p,q,r,s])
        print((p,q,r,s))

(0, 1, 2, 3)
(0, 1, 2, 5)
(0, 1, 2, 7)
(0, 1, 3, 2)
(0, 1, 3, 4)
(0, 1, 3, 6)
(0, 1, 4, 3)
(0, 1, 4, 5)
(0, 1, 4, 7)
(0, 1, 5, 2)
(0, 1, 5, 4)
(0, 1, 5, 6)
(0, 1, 6, 3)
(0, 1, 6, 5)
(0, 1, 6, 7)
(0, 1, 7, 2)
(0, 1, 7, 4)
(0, 1, 7, 6)
(0, 2, 4, 6)
(0, 2, 6, 4)
(0, 3, 1, 2)
(0, 3, 1, 4)
(0, 3, 1, 6)
(0, 3, 2, 1)
(0, 3, 2, 5)
(0, 3, 2, 7)
(0, 3, 4, 1)
(0, 3, 4, 5)
(0, 3, 4, 7)
(0, 3, 5, 2)
(0, 3, 5, 4)
(0, 3, 5, 6)
(0, 3, 6, 1)
(0, 3, 6, 5)
(0, 3, 6, 7)
(0, 3, 7, 2)
(0, 3, 7, 4)
(0, 3, 7, 6)
(0, 4, 2, 6)
(0, 4, 6, 2)
(0, 5, 1, 2)
(0, 5, 1, 4)
(0, 5, 1, 6)
(0, 5, 2, 1)
(0, 5, 2, 3)
(0, 5, 2, 7)
(0, 5, 3, 2)
(0, 5, 3, 4)
(0, 5, 3, 6)
(0, 5, 4, 1)
(0, 5, 4, 3)
(0, 5, 4, 7)
(0, 5, 6, 1)
(0, 5, 6, 3)
(0, 5, 6, 7)
(0, 5, 7, 2)
(0, 5, 7, 4)
(0, 5, 7, 6)
(0, 6, 2, 4)
(0, 6, 4, 2)
(0, 7, 1, 2)
(0, 7, 1, 4)
(0, 7, 1, 6)
(0, 7, 2, 1)
(0, 7, 2, 3)
(0, 7, 2, 5)
(0, 7, 3, 2)
(0, 7, 3, 4)
(0, 7, 3, 6)
(0, 7, 4, 1)
(0, 7, 4, 3)
(0, 7, 4, 5)
(0, 7, 5, 2)
(0, 7, 5, 4)
(0, 7, 5, 6)
(0, 7, 6, 1)
(0, 7, 6, 3)

In [17]:
len(doubles)

624

In [18]:
T1=[]
for p,q in singles:
    generator1=of.FermionOperator(((p, 1), (q, 0)))
    generator1+=of.hermitian_conjugated(generator1)*-1
    T1.append(generator1)

In [19]:
len(T1)

24

In [20]:
T2=[]
for p,q,r,s in doubles:
    generator2 =of.FermionOperator(((p, 1), (q, 1), (r, 0), (s, 0)))
    generator2+=of.hermitian_conjugated(generator2)*-1
    T2.append(generator2)

In [21]:
len(T2)

624

In [22]:
QT1=[]
for Op in T1:
    if (of.jordan_wigner(Op) not in QT1) and (of.jordan_wigner(Op)*-1 not in QT1):
        QT1.append(of.jordan_wigner(Op))
        
        
QT11=[]
for Op in T1:
        QT11.append(of.jordan_wigner(Op))        

In [23]:
QT2=[]
for Op2 in T2:
    if (of.jordan_wigner(Op2) not in QT2) and (of.jordan_wigner(Op2)*-1 not in QT2):
        QT2.append(of.jordan_wigner(Op2))
        
QT22=[]
for Op2 in T2:
        QT22.append(of.jordan_wigner(Op2))        

In [24]:
aq_in=[[0,0],[0,1],[1,0],[1,1]]
in_states=[]
for i in range(len(inputstates)):
    temp=inputstates[i]+aq_in[i]
    in_states.append(temp)

In [25]:
a_qubits=int(np.ceil(np.log2(k_number)))
w=[0]*2**(n_orbitals+a_qubits)
weight_in=[4,3,2,1]
for i in range(len(in_states)):
    w[binatodeci(in_states[i])]=weight_in[i]

v_in=npy.array(w)/npy.linalg.norm(np.array(w))   
N_qubits=n_orbitals

# auxiliary qubits
dev2 = qml.device("lightning.qubit", wires=n_orbitals+a_qubits)

input_array=np.array(v_in)

In [26]:
print(in_states)

[[1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 0, 1, 0, 0, 0, 1], [1, 1, 0, 1, 1, 0, 0, 0, 1, 0], [1, 0, 1, 1, 0, 1, 0, 0, 1, 1]]


In [27]:
@qml.qnode(dev2,diff_method='adjoint')
def pvqe(params):
    
    # QSP
    qml.QubitStateVector(np.array(input_array), wires=range(N_qubits+a_qubits))

    # modified UCCGSD
    for i in range(len(QT11)):
        H1_in=QT11[i]*1j
        qml.CommutingEvolution(qml.import_operator(H1_in),time=params[i])
    for k in range(len(QT22)):
        H2_in=QT22[k]*1j
        qml.CommutingEvolution(qml.import_operator(H2_in),time=params[len(QT11)+k])

    return qml.expval(Ham_in)

In [28]:
# Set up a optimizer, and run the pvqe
lr=0.03
b1=0.9
b2=0.999
opt = qml.AdamOptimizer(stepsize=lr,beta1=b1,beta2=b2)
max_iterations = 2000
conv_tol = 1e-8

print("Adam setting: ",'Learning rate: ',lr,'beta1: ',b1,'beta2: ',b2, 'eps: ',1e-8)

Adam setting:  Learning rate:  0.03 beta1:  0.9 beta2:  0.999 eps:  1e-08


In [29]:
E_energy_ext=v_in[binatodeci(in_states[0])]**2*E_ans_sz[0]+ \
v_in[binatodeci(in_states[1])]**2*E_ans_sz[1]+ \
v_in[binatodeci(in_states[2])]**2*E_ans_sz[2]+\
v_in[binatodeci(in_states[3])]**2*E_ans_sz[3]
print(E_energy_ext)

-1.8448493919999998


In [30]:
params = np.zeros(len(QT11)+len(QT22))
#params=np.zeros(shape)
pvqe(params)

array(-1.7822738)

In [31]:
EnsembleE=[]
#params = np.random.random(len(QT1)+len(QT2))
#params=np.zeros(len(QT1)+len(QT2))

params_set=[]
for n in range(max_iterations):
    params, prev_energy = opt.step_and_cost(pvqe, params)
    params_set.append(params)
    Energy=pvqe(params)
    EnsembleE.append(Energy)
    conv = np.abs(EnsembleE[-1] - prev_energy)

    if n % 10 == 0:
        print(f"Step = {n},  Ensemble Energy = {EnsembleE[-1]:.9f} Ha")
    if conv <= conv_tol:
        print(f"Step = {n},  Ensemble Energy = {EnsembleE[-1]:.9f} Ha")
        break

Step = 0,  Ensemble Energy = -0.875219141 Ha
Step = 10,  Ensemble Energy = -1.687519520 Ha
Step = 20,  Ensemble Energy = -1.787867031 Ha
Step = 30,  Ensemble Energy = -1.825446363 Ha
Step = 40,  Ensemble Energy = -1.837934067 Ha
Step = 50,  Ensemble Energy = -1.842534499 Ha
Step = 60,  Ensemble Energy = -1.843971044 Ha
Step = 70,  Ensemble Energy = -1.844511845 Ha
Step = 80,  Ensemble Energy = -1.844698146 Ha
Step = 90,  Ensemble Energy = -1.844791409 Ha
Step = 100,  Ensemble Energy = -1.844827416 Ha
Step = 110,  Ensemble Energy = -1.844841208 Ha
Step = 120,  Ensemble Energy = -1.844845964 Ha
Step = 122,  Ensemble Energy = -1.844846336 Ha


In [32]:
E_energy_ext=v_in[binatodeci(in_states[0])]**2*E_ans_sz[0]+ \
v_in[binatodeci(in_states[1])]**2*E_ans_sz[1]+ \
v_in[binatodeci(in_states[2])]**2*E_ans_sz[2]+\
v_in[binatodeci(in_states[3])]**2*E_ans_sz[3]
print("Exect ensemble: ", E_energy_ext)
print("Difference: ", pvqe(params_set[-1])-E_energy_ext)

Exect ensemble:  -1.8448493919999998
Difference:  3.0561692012653907e-06


In [33]:
dev3 = qml.device("lightning.qubit", wires=n_orbitals)

@qml.qnode(dev3,diff_method=None)
def pvqe_extract_state(ind_vector,params_in):
    
    # QSP
    qml.BasisState(np.array(ind_vector), wires=range(N_qubits))

    # modified UCCGSD
    for i in range(len(QT11)):
        H1_in=QT11[i]*1j
        qml.CommutingEvolution(qml.import_operator(H1_in),time=params_in[i])
    for k in range(len(QT22)):
        H2_in=QT22[k]*1j
        qml.CommutingEvolution(qml.import_operator(H2_in),time=params_in[len(QT11)+k]) 
    
    
    qml.Barrier(wires=range(N_qubits))
    return qml.state()


dev3 = qml.device("lightning.qubit", wires=n_orbitals)

@qml.qnode(dev3,diff_method=None)
def pvqe_extract(ind_vector,params_in):
    
    # QSP
    qml.BasisState(np.array(ind_vector), wires=range(N_qubits))
    # UCCGSD (Spin preserved)
    # modified UCCGSD
    # modified UCCGSD
    for i in range(len(QT11)):
        H1_in=QT11[i]*1j
        qml.CommutingEvolution(qml.import_operator(H1_in),time=params_in[i])
    for k in range(len(QT22)):
        H2_in=QT22[k]*1j
        qml.CommutingEvolution(qml.import_operator(H2_in),time=params_in[len(QT11)+k]) 
    qml.Barrier(wires=range(N_qubits))
    return qml.expval(Ham_in)

In [34]:
for i in range(len(inputstates)):
    print('Reference state: ', inputstates[i])
    id_energy=np.round(pvqe_extract(inputstates[i],params_set[-1]),8)
    #print('Final state: \n' ,np.round(pvqe_extract_state(inputstates[i],params_set[-1]),8))
    print("\nPVQE Energy_{}: ".format(i), id_energy," \nDifference: ",id_energy-E_ans_sz[i])
    S2= npy.transpose(pvqe_extract_state(inputstates[i],params_set[-1]) @ S2_Op @ pvqe_extract_state(inputstates[i],params_set[-1]))
    #print('index :',i)
    print('S^2: ', np.round(S2))
    print("S: ", np.round(-0.5+ npy.sqrt(1/4+S2)))
    Sz=npy.transpose(pvqe_extract_state(inputstates[i],params_set[-1]) @ S_z_Op @ pvqe_extract_state(inputstates[i],params_set[-1]))
    print('Sz: ', np.round(Sz),'\n \n')
    print('-----------------------------------------------------------------------------')

Reference state:  [1, 1, 1, 1, 0, 0, 0, 0]

PVQE Energy_0:  -2.10699594  
Difference:  9.799999998172382e-07
S^2:  0j
S:  0j
Sz:  0j 
 

-----------------------------------------------------------------------------
Reference state:  [1, 1, 1, 0, 0, 1, 0, 0]

PVQE Energy_1:  -1.65446918  
Difference:  1.710000000043621e-06
S^2:  (2+0j)
S:  (1+0j)
Sz:  0j 
 

-----------------------------------------------------------------------------
Reference state:  [1, 1, 0, 1, 1, 0, 0, 0]

PVQE Energy_2:  -1.36564953  
Difference:  3.140000000012577e-06
S^2:  0j
S:  0j
Sz:  0j 
 

-----------------------------------------------------------------------------
Reference state:  [1, 0, 1, 1, 0, 1, 0, 0]

PVQE Energy_3:  -1.28063428  
Difference:  4.807000000006667e-05
S^2:  (2+0j)
S:  (1+0j)
Sz:  0j 
 

-----------------------------------------------------------------------------


In [35]:
#save_file_name='LiH_{0}_{1}_params_3es.pickle'.format(sys_args,bond_length)
save_file_name='H4_full{0}_4es.pickle'.format(bond_length)
with open(save_file_name, 'wb') as f:
    pickle.dump(params_set, f)